<a href="https://colab.research.google.com/github/LeoFeatherstone/idPhyloWorkshop/blob/main/day2/BEAST_MCMC_WORKSHOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCMC - Phylodynamics @ The Doherty
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LeoFeatherstone/idPhyloWorkshop/blob/main/day2/BEAST_MCMC_WORKSHOP.ipynb)

This notebook shows how to run `beast` from the command line. Here we use [dynamic-beast](https://github.com/Wytamma/dynamic-beast) to modify the xml file and [beastiary](https://github.com/Wytamma/beastiary) to inspect the trace files in real-time.

Based on [EnzoAndree/ColabBEAST](https://github.com/EnzoAndree/ColabBEAST).

# Install requirements 

This group of cells will install all the requirements and set up the envrioment.

In [ ]:
#@title Check Nvidia GPU
import ipywidgets as widgets
import pandas as pd
import psutil
from time import sleep, time
from bokeh.resources import INLINE
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook, curdoc
from bokeh.themes import built_in_themes
from bokeh.models import BasicTickFormatter, Legend, NumeralTickFormatter
from pathlib import Path
import shlex, subprocess

def get_compute(name):
  if 'A100' in name:
    return '80'
  elif 'V100' in name:
    return '70'
  elif 'P4' in name:
    return '61'
  elif 'T4' in name:
    return '75'
  elif 'P100' in name:
    return '60'
  elif 'K80' in name:
    return '37'
def get_packagemanager_list(output):
  lines = outpack[4:]
  listofpack = []
  for package in lines:
    listofpack.append(package.split()[0])
  return listofpack

!nvidia-smi
Gcardinfo = !nvidia-smi --query-gpu=gpu_name,memory.total --format=csv,noheader,nounits
Gcardname, Gcardram = Gcardinfo[0].split(', ')

In [ ]:
%%capture
#@title Install BEAST2, Beagle, beastiary, and dynamic-beast
!pip install beastiary
!pip install dynamic-beast
!npm install -g localtunnel # used to serve beastiary over the internet
checkinstall = Path('./BEAST2Beagle_READY')
if not checkinstall.is_file():
    !wget https://github.com/CompEvol/beast2/releases/download/v2.6.4/BEAST.v2.6.4.Linux.tgz
    !tar -zxvf BEAST.v2.6.4.Linux.tgz
    !rm -fr BEAST.v2.6.4.Linux.tgz
    %cd /content
    !apt-get install build-essential autoconf automake libtool git pkg-config
    !git clone --depth=1 https://github.com/beagle-dev/beagle-lib.git
    %cd beagle-lib
    # http://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/
    # Nvidia A100 compute_80
    # Nvidia V100 compute_70
    # Nvidia P4 compute_61
    # Nvidia T4 compute_75
    # Nvidia P100 compute_60
    # Nvidia K80 compute_37
    !sed -i 's/-arch compute_30/-gencode=arch=compute_{get_compute(Gcardname)},code=sm_{get_compute(Gcardname)}/' configure.ac
    !./autogen.sh
    !./configure --prefix=$HOME
    !make install
    !make check
    %env LD_LIBRARY_PATH=/root/lib
    %cd /content
    !touch BEAST2Beagle_READY

In [ ]:
#@title Install BEAST2 modules
# !./beast/bin/packagemanager -add CoupledMCMC
# !./beast/bin/packagemanager -add bacter
# !./beast/bin/packagemanager -add NS
outpack = !./beast/bin/packagemanager -list
modlist = get_packagemanager_list(outpack)
typocheck = {x.lower(): x for x in modlist}
modules = '' #@param {type:"string"}
#@markdown - `modules` Specify the extra modules to be installed separated by commas. Leave it blank if you do not need extra modules.
#@markdown  - Use `!./beast/bin/packagemanager -list` to get a list of modules availables. 

to_install = []
if modules != '':
  errorfound = False
  modules = modules.split(',')
  modules = [m.strip() for m in modules]
  for m in modules:
    if m.lower() in typocheck.keys():
      to_install.append(typocheck[m.lower()])
    else:
      errorfound = True
      print(f'{m} is not found in the modulule list {modlist}')
      break
  if not errorfound:
    print(f'This modules will be installed: {to_install}')
    for m in to_install:
      !./beast/bin/packagemanager -add {m}
  


In [ ]:
#@title Check Beagle resources
!./beast/bin/beast -beagle_info 

# Run Beast

## Create a dynamic XML file

We will use `dynamic-beast` to create a dynamic version of the XML file. 


The `dynamic-beast` tool replaces all the parameter values in the XML file with the `$(id.key=value)` format. The value variable is the default value that was initially specified in the XML file. However, the value can be redefined when running a BEAST analysis by making use of the [BEAST2 definitions option](https://www.beast2.org/2021/03/31/command-line-options.html#-d) (`-D`) that allows for user specified values. 


In [ ]:
XML_FILE="MCMC.xml" # path to the xml file 
# Use dynamic beast to make a dynamic XML
!dynamic-beast --outfile dynamic_$XML_FILE $XML_FILE
!ls -l # should see dynmaic_MCMC.xml

Download the XML and look that dynamic parameters.

## Run a test analysis

A short run to check that everything is working before runing the full anlysis. 

- Set output dir to `test/`
- Set the MCMC chain length to 1 million 
- Collect 1000 samples

In [ ]:
DYNAMIC_XML="dynamic_MCMC.xml" # path to the dynamic XML file
OUTPUT_DIR="test/"
!mkdir -p $OUTPUT_DIR # the `!` tells jupyter to use bash 

CHAIN_LENGTH=1000000  # number of step in MCMC chain - int
NUMBER_OF_SAMPLES=1000  # number of samples to collect - int
SAMPLE_FREQUENCY=CHAIN_LENGTH // NUMBER_OF_SAMPLES  # calculate the frequency

# Format the dynamic variables
DYNAMIC_VARS=f"treelog.logEvery={SAMPLE_FREQUENCY},"
DYNAMIC_VARS+=f"tracelog.logEvery={SAMPLE_FREQUENCY},"
DYNAMIC_VARS+=f"mcmc.chainLength={CHAIN_LENGTH}"

# run beast
!nohup ./beast/bin/beast \
  -prefix $OUTPUT_DIR \
  -beagle \
  -beagle_GPU \
  -overwrite \
  -D "${DYNAMIC_VARS}" \
  $DYNAMIC_XML > $OUTPUT_DIR/beast.out  &
# The `nohup` and `&` at the end of the beast command tells bash to run the command in the
# background. The processes can be stopped with the `ps` and `kill` commands.

## Inspect the test run with beasity 

Wait for the `.log` file to appear before running this cell.

In [ ]:
#@title Inspect the log file [beastiary](https://github.com/Wytamma/beastiary) 

from IPython.display import Javascript
import os
import signal
from time import sleep
from glob import glob
import socket

path_to_log_files = '/content/test/*.log' #@param {type:"string"}

# stop beastiary if it's already running 
try:
  beastiary_proc = !ps | grep 'beastiary$' 
  beastiary_pid = int(beastiary_proc[0].split()[0])
  os.kill(beastiary_pid, 9)
except:
  pass

s = socket.socket()
s.bind(("", 0))
PORT = s.getsockname()[1]
s.close()

print("\n🐙🐁 STARTING BEASTIARY 🐁🐙\n")
cli = f'beastiary --no-security --port {PORT} {path_to_log_files} &> beastiary.logger &'
get_ipython().system_raw(cli) 

# wait for server to start
sleep(5)

!lt --port $PORT --subdomain beastiary

## Run the full analysis 

- Set the starting value of clock to 3×10−3 subs/site/year 
- Sample from the prior (Prior predictive checking)
- Run the analysis x 2 (10E7 states and 10,000 samples)
- Save the seed for each analysis

In [ ]:
import random # to create a seed

DYNAMIC_XML = "dynamic_MCMC.xml" # path to the dynamic XML file
OUTPUT_DIR = "full/"
!mkdir -p $OUTPUT_DIR 

# Dynamic variables
DYNAMIC_VARS = '"clockRate.c:NorthAm=0.003"'

# --- Prior predictive checking ---
RUN_DIR = f"{OUTPUT_DIR}/prior/"
!mkdir -p $RUN_DIR 
PREFIX = f"{RUN_DIR}/prior."

# Make seed and save the seed
SEED = random.randint(0, 10000) 
!echo $SEED > $RUN_DIR/seed.txt # save the seed

!nohup ./beast/bin/beast \
  -prefix $PREFIX \
  -statefile $PREFIX\state \
  -seed $SEED \
  -beagle \
  -beagle_GPU \
  -overwrite \
  -sampleFromPrior \
  -D $DYNAMIC_VARS \
  $DYNAMIC_XML > $PREFIX\beast.out  &

# --- Full Analysis --- 
NUMBER_OF_REPEATS = 2  # run multiple times to test for convergence

for i in range(NUMBER_OF_REPEATS):
  # set up output dir
  RUN_DIR = f"{OUTPUT_DIR}/{i}/"
  !mkdir -p $RUN_DIR 
  PREFIX = f"{RUN_DIR}/{i}."

  # Make seed and save the seed
  SEED = random.randint(0, 10000) 
  !echo $SEED > $RUN_DIR/seed.txt 

  # run beast
  !nohup ./beast/bin/beast \
    -prefix $PREFIX \
    -statefile $PREFIX\state \
    -beagle \
    -seed $SEED \
    -beagle_GPU \
    -overwrite \
    -D $DYNAMIC_VARS \
    -DFout $OUTPUT_DIR\beast.xml \
    $DYNAMIC_XML > $PREFIX\beast.out  &



## Inspect the full run with beasity 

Wait for the `.log` files to appear before running this cell.

In [ ]:
#@title Inspect the log file [beastiary](https://github.com/Wytamma/beastiary) 

from IPython.display import Javascript
import os
import signal
from time import sleep
from glob import glob
import socket

path_to_log_files = '/content/full/*/*.log' #@param {type:"string"}

# stop beastiary if it's already running 
try:
  beastiary_proc = !ps | grep 'beastiary$' 
  beastiary_pid = int(beastiary_proc[0].split()[0])
  os.kill(beastiary_pid, 9)
except:
  pass

s = socket.socket()
s.bind(("", 0))
PORT = s.getsockname()[1]
s.close()

print("\n🐙🐁 STARTING BEASTIARY 🐁🐙\n")
cli = f'beastiary --no-security --port {PORT} {path_to_log_files} &> beastiary.logger &'
get_ipython().system_raw(cli) 

# wait for server to start
sleep(5)

!lt --port $PORT --subdomain beastiary